In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install konlpy
!pip install nltk
!pip install sentence_transformers

# 연극+뮤지컬

In [ ]:
import pandas as pd

def concat_play_musical(play, musical):
  try:
    playdf = pd.read_excel(play)
    musicaldf = pd.read_excel(musical)
  except:
    playdf = pd.read_csv(play)
    musicaldf = pd.read_csv(musical)
  df = pd.concat([playdf, musicaldf])
  df = df.drop(['Unnamed: 0'], axis=1)
  df = df.reset_index(drop=True)
  return df

# 파일 경로를 함수화하면, 모든 코드가 복잡해지므로 전역변수화 시켰음
play_top = "/content/drive/MyDrive/캡스톤/데이터/연극/연극_TOP77_장소병합.xlsx"
musical_top = "/content/drive/MyDrive/캡스톤/데이터/뮤지컬/뮤지컬_TOP77.xlsx"
play_top_star = "/content/drive/MyDrive/캡스톤/데이터/연극/playTop77_star.xlsx"
musical_top_star = "/content/drive/MyDrive/캡스톤/데이터/뮤지컬/musicalTop77_star.xlsx"
play_actor_lank = "/content/drive/MyDrive/캡스톤/데이터/연극/연극 배우 랭킹.csv"
musical_actor_lank = "/content/drive/MyDrive/캡스톤/데이터/뮤지컬/뮤지컬 배우 랭킹.csv"
play_top_cast = "/content/drive/MyDrive/캡스톤/데이터/연극/playTop77_casting.xlsx"
musical_top_cast = "/content/drive/MyDrive/캡스톤/데이터/뮤지컬/musicalTop77_casting.xlsx"
play_top_poster = '/content/drive/MyDrive/캡스톤/데이터/연극/연극_TOP77_장소병합 (+포스터).xlsx'
musical_top_poster = '/content/drive/MyDrive/캡스톤/데이터/뮤지컬/뮤지컬_TOP77 (+포스터).xlsx'
play_stat = '/content/drive/MyDrive/캡스톤/데이터/연극/연극_통계_e.xlsx'
musical_stat = '/content/drive/MyDrive/캡스톤/데이터/뮤지컬/뮤지컬_통계_e.xlsx'

top_lank = concat_play_musical(play_top, musical_top)
top_star = concat_play_musical(play_top_star, musical_top_star)
actor_lank = concat_play_musical(play_actor_lank, musical_actor_lank)
top_cast = concat_play_musical(play_top_cast, musical_top_cast)

top_poster = concat_play_musical(play_top_poster, musical_top_poster)
top_poster = top_poster.drop('Image', axis=1)

stat = concat_play_musical(play_stat, musical_stat)
stat = stat.drop('Title', axis=1)

# 컨텐츠 기반 필터링

## 장르

In [ ]:
import os
import pandas as pd
import seaborn as sns
import numpy as np
import random
from matplotlib import pyplot as plt
from datetime import datetime
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

def genre_similarlity(top70_df, title_name):
  df=top70_df[['Title', '장르']]
  from sklearn.feature_extraction.text import TfidfVectorizer

  #  전처리 - 공백 제거하기, ',' 제거
  df['장르']=df['장르'].str.replace(' ', '').str.replace(',', ', ')
  df['genres_literal']=df['장르'].str.split(',')
  df['genres_literal'] = df['genres_literal'].apply(lambda x : (' ').join(x))

  tfidf_vect = TfidfVectorizer()
  genre_mat1=tfidf_vect.fit_transform(df['genres_literal'])

  from sklearn.metrics.pairwise import cosine_similarity

  genre_sim1 = cosine_similarity(genre_mat1, genre_mat1)
  genre_sim_sorted_ind1 = genre_sim1.argsort()[:, ::-1]

  #  movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame 추출
  title_movie = df[df['Title'] == title_name]

  # 유사도 순으로 top_n 개의 index 추출
  title_index = title_movie.index.values
  similar_indexes = genre_sim_sorted_ind1[title_index, :]

  similar_indexes = similar_indexes.reshape(-1)

  # 유사한 영화 중에 유사도가 높은 순으로 정렬한 인덱스에 대응되는 유사도 추출
  similar_scores = genre_sim1[title_index, similar_indexes]
  similar_movie_df = df.iloc[similar_indexes]
  similar_movie_df['Cosine_Similarity'] = similar_scores

  # 코사인 유사도 계산
  similar_movie_df['Cosine_Similarity'] = similar_movie_df['Cosine_Similarity'].round(2)

  similar_movie_df = similar_movie_df[['Title', 'Cosine_Similarity']]
  similar_movie_df = similar_movie_df[similar_movie_df['Title'] != title_name]

  return similar_movie_df

def genre_sim(title):
  similar_df = genre_similarlity(top_lank, title)
  return similar_df

## 줄거리 + 이미지

In [ ]:
import nltk
nltk.download('punkt')

import torch
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
from konlpy.tag import Okt, Kkma, Komoran
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

import networkx
from tqdm.autonotebook import tqdm
from tqdm import tqdm, tqdm_notebook

import seaborn as sns
import matplotlib.pyplot as plt

from gensim.models import word2vec
import itertools

from PIL import Image
import os
from tqdm import tqdm, tqdm_notebook
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from torch.utils.data import Dataset

import torchvision.transforms as T
import torch.nn as nn
import torch.optim as optim

import tensorflow as tf
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications import vgg16
from keras.models import Model
from tensorflow.keras.utils import load_img, img_to_array
from sklearn.metrics.pairwise import cosine_similarity
from numpy.linalg import norm
from numpy.linalg import norm
import math
import time
import pickle
import unicodedata


model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3), pooling='max')
embedder = SentenceTransformer('jhgan/ko-sroberta-multitask')
embedder.max_seq_len = 128
extensions = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG', '.gif']

def plot_vector(df, embedder):
  df['줄거리'] = df['줄거리'].apply(lambda x: re.sub(r'[^가-힣a-zA-Z\s]', '', x))
  stories = df['줄거리'].tolist()
  story_embeddings = embedder.encode(stories, convert_to_tensor=True)
  xx = pd.DataFrame(story_embeddings)
  return xx

imgs_path = "/content/drive/MyDrive/캡스톤/데이터/포스터/"
imgs_model_width, imgs_model_height = 224, 224

nb_closest_images = 10

# VGG16 모델 로드
vgg_model = vgg16.VGG16(weights='imagenet')

# 마지막 레이어를 제거하여 예측 대신 특징을 얻기
img_features = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer("fc2").output)

extensions = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG', '.gif']
files = [imgs_path + x for x in os.listdir(imgs_path) if any(ext in x for ext in extensions)]

importedImages = []

filenames = []
feature_list = []

for f in tqdm_notebook(files):
    filename = f
    original = load_img(filename, target_size=(imgs_model_width, imgs_model_height))
    numpy_image = img_to_array(original)
    image_batch = np.expand_dims(numpy_image, axis=0)

    importedImages.append(image_batch)
    filenames.append(filename)

images = np.vstack(importedImages)

processed_imgs = preprocess_input(images.copy())
imgs_features = img_features.predict(processed_imgs)
filenames, feature_list = filenames, imgs_features.tolist()
x = pd.DataFrame(feature_list)

# 줄거리와 포스터 이미지 벡터 concat
def p_i_main(model):
  pv = plot_vector(top_poster,embedder)
  iv = pd.DataFrame(feature_list)
  plot_image_v = pd.concat([pv, iv], axis=1)
  return plot_image_v

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


553467096/553467096 [==============================] - 16s 0us/step


  0%|          | 0/127 [00:00<?, ?it/s]

4/4 [==============================] - 82s 20s/step


In [ ]:
plot_image_v = p_i_main(model)
top_stat = pd.concat([plot_image_v, stat], axis=1)

In [ ]:
import pickle
def load_pickle(filename):
    with open(filename, 'rb') as f:
        res = pickle.load(f)
    return res

plot_image_v = load_pickle('/content/drive/MyDrive/캡스톤/데이터/sbert_vgg16_vecs.pickle')
top_stat = pd.concat([plot_image_v, stat], axis=1)

In [ ]:
def plot_image_sim(title):
  target_index = top_poster[top_poster['Title'] == title].index[0]

  # 코사인 유사도
  cosine_sim = cosine_similarity(top_stat)
  # 특정 작품과 다른 작품들 간의 유사도
  target_similarities = cosine_sim[target_index]
  plot_image_weight = pd.DataFrame([top_poster['Title'], target_similarities], index=['Title', 'Similarity']).T

  return plot_image_weight

## 별점

In [ ]:
# user_ratings : 별점리스트, review_count: 전체 행개수
def calculate_weighted_rating (user_ratings, review_count, base_weight=1.0):

  # 가중치 계산
  weight = base_weight*( 1.0 + review_count)    # 사람 수가 적을수록 가중치 감소/ 기본 가중치값 먼저 필요

  # 평균 별점 계산
  average_rating = sum(user_ratings)/len(user_ratings)   # 작품명 같은 것끼리 계산

  # 가중 평균 별점 계산
  weighted_rating = average_rating*weight

  return weighted_rating

def star_weight():
  import pandas as pd
  star_df = top_star
  star = star_df['Star']
  num_play = star_df.shape[0]

  # 제목별로 별점 그룹화
  grouped_title = star_df.groupby('Title')['Star'].apply(list).reset_index()
  type(grouped_title)

  # 별점 가중치 함수 데이터에 적용

  review_count = 100
  weighted_ratings = {}

  for row in grouped_title.itertuples():
    title = row.Title
    user_ratings = row.Star

    # 가중 평균 계산 함수 호출
    weighted_rating = calculate_weighted_rating(user_ratings, review_count=len(user_ratings))
    weighted_ratings[title] = weighted_rating

  for title, weighted_rating in weighted_ratings.items():
      print(f"Title: {title}, Weighted Rating: {weighted_rating}")
  return weighted_ratings

weighted_ratings = star_weight()

Title: (리얼타임 코믹연극) 택시안에서 - 서울, Weighted Rating: 1005.744075829384
Title: (코믹연극) 달동네-부산, Weighted Rating: 73.92857142857143
Title: 10년 연속 1위 연극〈옥탑방고양이〉- 틴틴홀, Weighted Rating: 2462.819607843137
Title: 2023 연극 〈친정엄마와 2박 3일〉 - 고양, Weighted Rating: 33.142857142857146
Title: 2시간탈출 졸탄쇼, Weighted Rating: 653.7720588235295
Title: 3대가 웃고 우는 연극 〈염쟁이 유씨〉, Weighted Rating: 68.92307692307693
Title: 4D공포연극 〈스위치〉, Weighted Rating: 2457.9555555555557
Title: ★평점9.5★ 코미디의맛/쇼미더퍼니, Weighted Rating: 2124.7640449438204
Title: 공포스릴러연극〈두여자〉- 대구, Weighted Rating: 310.0
Title: 공포연극 조각, Weighted Rating: 617.7890625
Title: 공포연극 ［자취］, Weighted Rating: 966.81
Title: 국민 코믹 연극 〈오백에삼십〉 - 대학로 세우아트센터 1관, Weighted Rating: 2800.8966725043783
Title: 그곳에 있었다, Weighted Rating: 57.81818181818182
Title: 나의 장례식에 와줘, Weighted Rating: 73.92857142857143
Title: 난타(NANTA) - 명동공연, Weighted Rating: 790.8518518518518
Title: 대학로 1위 연극 〈쉬어매드니스〉, Weighted Rating: 2432.7607843137253
Title: 대학로 청소년연극 〈사춘기메들리〉, Weighted Rating: 716.9444444444

## 배우

In [ ]:
import pandas as pd
def actor_lanking(actor_csv):
  actor_csv = actor_csv.reset_index()
  actor_csv.columns = ["index", "actor_name", "masterpiece"]

  # 랭킹에 있는 배우들의 작품만 따로 추출 (작품명이 \n으로 이어진 하나의 문자열이기 때문에, 리스트화 시킬 예정)
  actor_mp = actor_csv['masterpiece'].to_list()

  actor_dic = actor_csv.set_index('index').T.to_dict()  # 딕셔너리 형태로 변환

  # key: actor_name, value: index, masterpiece
  for key, value in actor_dic.items():
    temp = value['masterpiece']
    temp2 = temp.split('\n')
    value['masterpiece'] = temp2
  return actor_dic

def cast_actor(inter_actor):
  inter_actor = inter_actor[['Title', 'cast_act']]

  # 문자열 형태로 되어 있는 cast_act 열을 리스트화 시켜서 다시 집어 넣을 예정
  inter_actor_cast = inter_actor['cast_act']

  inter_list = []
  for i in range(len(inter_actor)):
    string = inter_actor_cast[i]
    ch_string = string.split(', ')
    inter_list.append(ch_string)

  # Title 기준 딕셔너리화
  inter_dic = inter_actor.set_index('Title').T.to_dict()

  # key: Title, cast_act
  i = 0
  for key, value in inter_dic.items():
    value['cast_act'] = inter_list[i]
    i+=1
  return inter_dic, inter_actor

# 배우 이름 찾기 (랭킹 도출) 함수
def find_value(dictionary, finding_value):
    for key, value in dictionary.items():
        if (finding_value.find(value['actor_name']) > -1):
            return key

def search_actor(inter_dic, actor_dic):
  actor_mp = {}
  i = 0
  for key1, value1 in inter_dic.items():
      for name in value1['cast_act']:
          index = find_value(actor_dic, name)
          if index is not None:
              for key2, value2 in actor_dic.items():
                  if key2 == index:
                    # 여기서부터 짰음
                      recent_performances = value2['masterpiece']

                      for title in recent_performances:
                        title = title.split('(')
                        title = title[0]

                        # 인터파크 제목에서 최근 공연을 찾음
                        temp = key1.find(title)
                        if temp > -1:
                          print(f"배우 {name}의 최근 공연 '{key1}'이 인터파크 제목에 포함됨.")
                          actor_mp[i] = {'actor' : name, 'Title' : key1}
                          i+=1
  return actor_mp

def actor_name_dic(actor_dic, inter_dic, SA):
  dic = {}
  for key1, value1 in actor_dic.items():
    dic[key1] = value1['actor_name']


  index = 200  # 순위권 이후 배우 추가용
  for key1, value1 in inter_dic.items():
    for i in value1['cast_act']:  # 공연별 캐스팅 배우 딕셔너리화
      ret = 0 # 순위권 배우 이름 중복 추가 방지용
      for key2, value2 in SA.items(): # 순위권 배우 판별
        if (value2['Title'] == key1 and value2['actor'] == i) or (i == '해당없음'): # 순위권 배우거나, 배우명이 없는 경우 추가 제외
          ret = 1
          break
      if(ret == 0):
        for ran in range(100, index):  # 동명이인 찾기
          if(dic[ran] == i ):
            ret = 1
            break
          else:
            ret = 0

      if(ret == 0): # 그 외 전부 추가
        dic[index] = i
        index += 1
  dic_t = {v:k for k,v in dic.items()}
  return dic, dic_t

def change_index(search, dic):
  ret_dic = {}
  for key, value in search.items():
    index = dic[value['actor']]
    ret_dic[index] = value['Title']
  return ret_dic

def cast_list(dic1, dic2, dic_t):
  cast_dic = {}
  for key1, value1 in dic2.items():
    cast_act = value1['cast_act']
    cast_index = []
    # 배우 인덱스 저장
    for name in cast_act:
      cast_index.append(dic_t.get(name))
    cast_dic[key1] = cast_index
  return cast_dic

def actor_rate(cast_dic, title_name, ci):
  score_list = []
  cast_list = cast_dic[title_name]
  for key, value in cast_dic.items():
    score = 0
    for index in cast_list:
      # 이 공연의 배우가 있을 경우 추천점수 +1점
      if((key != title_name) and (index in value)):
        score += 1
      # 유명한 배우가 있을 경우 추천점수 +2.5점
    for name, title in ci.items():
      if(key == title):
        score += 2.5
    score_list.append(score)
  return score_list

actor_dic = actor_lanking(actor_lank)
inter_dic, inter_actor = cast_actor(top_cast)
SA = search_actor(inter_dic, actor_dic)
pt_actor_dic, pt_actor_dic_t = actor_name_dic(actor_dic, inter_dic, SA)
ci = change_index(SA, pt_actor_dic_t) # 랭킹배우의 이름->인덱스


배우 박은영의 최근 공연 '뮤직드라마 〈불편한 편의점〉'이 인터파크 제목에 포함됨.
배우 최덕문의 최근 공연 '연극 〈기형도 플레이〉'이 인터파크 제목에 포함됨.
배우 박호산의 최근 공연 '연극 〈기형도 플레이〉'이 인터파크 제목에 포함됨.
배우 이재환의 최근 공연 '뮤지컬 〈22년 2개월〉'이 인터파크 제목에 포함됨.
배우 정민의 최근 공연 '뮤지컬 〈구텐버그〉'이 인터파크 제목에 포함됨.
배우 김지현의 최근 공연 '뮤지컬 〈그날들〉 10주년 기념 공연 - 대전'이 인터파크 제목에 포함됨.
배우 린아의 최근 공연 '뮤지컬 〈레미제라블〉 - 부산'이 인터파크 제목에 포함됨.
배우 정민의 최근 공연 '뮤지컬 〈스모크〉'이 인터파크 제목에 포함됨.
배우 김신의의 최근 공연 '뮤지컬 〈삼총사〉'이 인터파크 제목에 포함됨.
배우 김신의의 최근 공연 '뮤지컬 〈삼총사〉'이 인터파크 제목에 포함됨.
배우 송원근의 최근 공연 '뮤지컬 〈오페라의 유령〉 - 서울'이 인터파크 제목에 포함됨.
배우 김아선의 최근 공연 '뮤지컬 〈오페라의 유령〉 - 서울'이 인터파크 제목에 포함됨.
배우 차지연의 최근 공연 '뮤지컬 〈컴프롬어웨이〉'이 인터파크 제목에 포함됨.
배우 문혜원의 최근 공연 '뮤지컬 〈타오르는 어둠 속에서〉'이 인터파크 제목에 포함됨.
배우 송상은의 최근 공연 '뮤지컬 〈블랙메리포핀스〉'이 인터파크 제목에 포함됨.
배우 규현의 최근 공연 '뮤지컬 ＇벤허＇'이 인터파크 제목에 포함됨.
배우 이희정의 최근 공연 '뮤지컬 ＇벤허＇'이 인터파크 제목에 포함됨.
배우 이자람의 최근 공연 '창작가무극 〈순신〉'이 인터파크 제목에 포함됨.
배우 고미경의 최근 공연 '창작가무극 〈순신〉'이 인터파크 제목에 포함됨.


## 가중치 총합

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler((0,5))

def weight_calc(weighted_ratings):
  weighted_df = pd.DataFrame(weighted_ratings, index = ['StarScore']).T
  weighted_df[:] = scaler.fit_transform(weighted_df[:])
  return weighted_df

def castscore_calc(title, actor_dic, inter_dic, actor_dic_t, ci, inter_actor):
  cast_dic = cast_list(actor_dic, inter_dic, actor_dic_t)
  score_list = actor_rate(cast_dic, title, ci)
  inter_actor['actor_score'] = score_list
  casting = inter_actor.sort_values(by=['actor_score'], ascending=False)
  casting = casting[['Title', 'actor_score']]
  casting = casting.reset_index(drop=True)

  cast = {}
  for i in range(len(casting)):
    key = casting['Title'][i]
    value = casting['actor_score'][i]
    cast[key] = value
    castscore_df = pd.DataFrame(cast, index = ['CastScore']).T
    castscore_df[:] = scaler.fit_transform(castscore_df[:])

  return castscore_df

def genre_calc(title, genre_sim):
  genre_weight = genre_sim.rename(columns = {'Title' : 'Title', 'Cosine_Similarity' : 'GenreScore'})
  genre_weight = genre_weight.set_index(keys=['Title'])
  genre_weight[:] = scaler.fit_transform(genre_weight[:])
  return genre_weight

def plot_image_calc(title):
  plot_image_weight = plot_image_sim(title)
  pi_weight = plot_image_weight.rename(columns = {'Title':'Title', 'Similarity':'PIScore'})
  pi_weight = pi_weight.set_index(keys=['Title'])
  pi_weight = pi_weight.sort_values(by=['PIScore'], ascending=False)
  pi_weight = pi_weight.iloc[1:]
  pi_weight[:] = scaler.fit_transform(pi_weight[:])
  return pi_weight

# 줄거리+이미지 / 장르
def merge_calc_pi(weighted_df, castscore_df, pi_df, genre_df, pm = 0):
  func_list = []
  w_df = pd.concat([weighted_df, castscore_df, pi_df, genre_df], axis=1, join='inner')
  for row in w_df.iterrows():
    #print(row[1]['Weight'])
    ss = row[1]['StarScore']
    cs = row[1]['CastScore']
    ps = row[1]['PIScore']
    gs = row[1]['GenreScore']
    if (pm == 0): # 연극일 경우
      func = gs * 0.35 + ps * 0.3 + cs * 0.2 + ss * 0.15
    else:  # 뮤지컬일 경우
      func = cs * 0.35 + gs * 0.3 + ps * 0.2 + ss * 0.15
    func_list.append([row[0], func])
    #print(f"Title: {row[0]}, Weighted Rating: {func}")
  return func_list

def score_pg(title):
  weighted_df = weight_calc(weighted_ratings)
  castscore_df = castscore_calc(title, pt_actor_dic, inter_dic, pt_actor_dic_t, ci, inter_actor)
  pi_df = plot_image_calc(title)
  genre_sm = genre_sim(title)
  genre_df = genre_calc(title, genre_sm)
  func_list = merge_calc_pi(weighted_df, castscore_df, pi_df, genre_df)
  play_score = pd.DataFrame(func_list, columns=['Title', 'Score']).sort_values('Score', ascending=False)
  return play_score

# 협업 필터링

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적 곱으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)

    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]

    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]

    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse

def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda = 0.01):
    num_users, num_items = R.shape
    # P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 랜덤한 값으로 입력합니다.
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))

    break_count = 0

    # R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장.
    non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]

    # SGD기법으로 P와 Q 매트릭스를 계속 업데이트.
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값 구함
            eij = r - np.dot(P[i, :], Q[j, :].T)
            # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])

        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10) == 0 :
            print("### iteration step : ", step," rmse : ", rmse)

    return P, Q

# 성능평가를 위해 return 값을 unseen_list > movies_list로 변경
def get_unseen_movies(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 영화정보 추출하여 Series로 반환함.
    # 반환된 user_rating 은 영화명(title)을 index로 가지는 Series 객체임.
    user_rating = ratings_matrix.loc[userId,:]

    # user_rating이 0보다 크면 기존에 관람한 영화임. 대상 index를 추출하여 list 객체로 만듬
    already_seen = user_rating[ user_rating > 0].index.tolist()

    # 모든 영화명을 list 객체로 만듬.
    movies_list = ratings_matrix.columns.tolist()

    # list comprehension으로 already_seen에 해당하는 movie는 movies_list에서 제외함.
    unseen_list = [ movie for movie in movies_list if movie not in already_seen]

    return movies_list

def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자id index와 unseen_list로 들어온 영화명 컬럼을 추출하여
    # 가장 예측 평점이 높은 순으로 정렬
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

total_star = top_star.reset_index(drop=True)
# 동일한 ID가 동일한 Title에 대해서 남긴 평점은 평균값으로 처리
pivot_df = total_star.pivot_table(index='ID', columns='Title', values='Star', aggfunc='mean')
# *** 아이디 삭제
pivot_df = pivot_df.drop(pivot_df.index[0])
pivot_df = pivot_df.fillna(0)
P, Q = matrix_factorization(pivot_df.values, K=50, steps=200, learning_rate=0.01, r_lambda = 0.01)
pred_matrix = np.dot(P, Q.T)
ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index= pivot_df.index,
                                   columns = pivot_df.columns)

### iteration step :  0  rmse :  4.859813043156199
### iteration step :  10  rmse :  0.4747636218515914
### iteration step :  20  rmse :  0.15651240858795534
### iteration step :  30  rmse :  0.08953775817763482
### iteration step :  40  rmse :  0.06525053793574208
### iteration step :  50  rmse :  0.05817221532940333
### iteration step :  60  rmse :  0.05354357339339101
### iteration step :  70  rmse :  0.04992442462248396
### iteration step :  80  rmse :  0.04716792049858878
### iteration step :  90  rmse :  0.04500818518431909
### iteration step :  100  rmse :  0.043214745716051856
### iteration step :  110  rmse :  0.04165723965784072
### iteration step :  120  rmse :  0.0402710046137485
### iteration step :  130  rmse :  0.039021327807607115
### iteration step :  140  rmse :  0.03788610667928627
### iteration step :  150  rmse :  0.03684913435070453
### iteration step :  160  rmse :  0.03589756463243925
### iteration step :  170  rmse :  0.035020835200331175
### iteration step :  

# 다 돌리고, 메인 함수에서 score_list와 collab_list만 수정할 것
* MinMaxScaler 만 수정하면 될 것!!

# 메인

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# 컨텐츠 (장르 / 줄거리+이미지)
def score_list(high_rated_titles):
  # 사용자의 가장 높은 평점 작품 : score 함수 => 합산
  scaler = MinMaxScaler((0,0.4))
  scaler_10 = MinMaxScaler((0,10))
  df = pd.DataFrame()
  for title in high_rated_titles:
    score_df = score_pg(title)
    df = pd.concat([df, score_df])
  df1 = df.groupby('Title')['Score'].sum()
  df1 = df1.sort_values(axis=0, ascending=False)
  df = df1.to_frame()
  df[:] = scaler.fit_transform(df[:])
  return df

# 협업
def collab_list(userid):
  scaler = MinMaxScaler((0,0.6))
  scaler_10 = MinMaxScaler((0,10))
  # user의 가장 높은 평점 찾기, 해당 타이틀 추출
  total_star = top_star.reset_index(drop=True)
  watched_titles = total_star[total_star['ID'] == userid]
  max_rating = watched_titles['Star'].max()
  high_rated_titles = watched_titles[watched_titles['Star']== max_rating]['Title'].unique()
  # 협업필터링
  unseen_list = get_unseen_movies(pivot_df, userid)
  predicted_movies = recomm_movie_by_userid(ratings_pred_matrix, userid, unseen_list, top_n=130)

  ## 평점 데이타를 DataFrame으로 생성.
  recomm_movies = pd.DataFrame(data=predicted_movies.values,index=predicted_movies.index,columns=['pred_score'])
  recomm_movies_idx  = recomm_movies.reset_index()
  recomm_movies_idx = recomm_movies_idx.set_index(keys=['Title'])
  recomm_movies_idx[:] = scaler.fit_transform(recomm_movies_idx[:])

  #recomm_movies_idx['score'] = list(map(lambda x: 10-x, range(10)))
  return recomm_movies_idx, high_rated_titles


def mat_content_image(userid):
  recomm_movies_idx, high_rated_titles = collab_list(userid)
  # 컨텐츠 기반 합산
  scores_df = score_list(high_rated_titles)

  total_df = pd.merge(scores_df, recomm_movies_idx, left_index=True, right_index=True, how='outer')
  total_df['score'] = total_df['Score'] + total_df['pred_score']
  total_df = total_df[['score']]

  total_score = total_df.sort_values('score', ascending=False).head(10)

  return total_score

def find_title(top_lank, title_input):
  title_str = title_input.replace(" ", "")
  title_strip = top_lank['Title'].str.replace(" ", "")
  title_index = title_strip[title_strip.str.contains(title_str)].index[0]
  return top_lank.loc[title_index]['Title']

if __name__ == '__main__':
  userid = 'devil***'
  result = mat_content_image(userid)

In [ ]:
def score_list(high_rated_titles):
  # 사용자의 가장 높은 평점 작품 : score 함수 => 합산
  scaler = MinMaxScaler((0,0.7))
  scaler_10 = MinMaxScaler((0,10))
  df = pd.DataFrame()
  for title in high_rated_titles:
    score_df = score_pg(title)
    df = pd.concat([df, score_df])
  df1 = df.groupby('Title')['Score'].sum()
  df1 = df1.sort_values(axis=0, ascending=False)
  df = df1.to_frame()
  df[:] = scaler.fit_transform(df[:])
  return df

# 협업
def collab_list(userid):
  scaler = MinMaxScaler((0,0.3))
  scaler_10 = MinMaxScaler((0,10))
  # user의 가장 높은 평점 찾기, 해당 타이틀 추출
  total_star = top_star.reset_index(drop=True)
  watched_titles = total_star[total_star['ID'] == userid]
  max_rating = watched_titles['Star'].max()
  high_rated_titles = watched_titles[watched_titles['Star']== max_rating]['Title'].unique()
  # 협업필터링
  unseen_list = get_unseen_movies(pivot_df, userid)
  predicted_movies = recomm_movie_by_userid(ratings_pred_matrix, userid, unseen_list, top_n=130)

  ## 평점 데이터를 DataFrame으로 생성.
  recomm_movies = pd.DataFrame(data=predicted_movies.values,index=predicted_movies.index,columns=['pred_score'])
  recomm_movies_idx  = recomm_movies.reset_index()
  recomm_movies_idx = recomm_movies_idx.set_index(keys=['Title'])
  recomm_movies_idx[:] = scaler.fit_transform(recomm_movies_idx[:])

  #recomm_movies_idx['score'] = list(map(lambda x: 10-x, range(10)))
  return recomm_movies_idx, high_rated_titles


def mat_content_image(userid):
  recomm_movies_idx, high_rated_titles = collab_list(userid)
  # 컨텐츠 기반 합산
  scores_df = score_list(high_rated_titles)

  total_df = pd.merge(scores_df, recomm_movies_idx, left_index=True, right_index=True, how='outer')
  total_df['score'] = total_df['Score'] + total_df['pred_score']
  total_df = total_df[['score']]

  total_score = total_df.sort_values('score', ascending=False).head(10)

  return total_score

def find_title(top_lank, title_input):
  title_str = title_input.replace(" ", "")
  title_strip = top_lank['Title'].str.replace(" ", "")
  title_index = title_strip[title_strip.str.contains(title_str)].index[0]
  return top_lank.loc[title_index]['Title']

def predict(userid):
   result = mat_content_image(userid)
   df = pd.read_excel('/content/drive/MyDrive/캡스톤/데이터/연극+뮤지컬 (+포스터).xlsx')
   df = df.drop(df.columns[0], axis=1)
   result['Image'] = result.index.map(lambda x: df[df['Title'] == find_title(df, x)]['Image'].iloc[0])
   return result

In [ ]:
predict('devil***')

,score,Image
Title,,
뮤지컬 〈타오르는 어둠 속에서〉,0.839626,뮤지컬 〈타오르는 어둠 속에서〉.png
뮤지컬 ＇벤허＇,0.756165,뮤지컬 ＇벤허＇.png
뮤지컬 〈블랙메리포핀스〉,0.740038,뮤지컬 〈블랙메리포핀스〉.png
코믹감동 휴먼판타지 연극 〈2호선 세입자〉,0.716521,코믹감동 휴먼판타지 연극 〈2호선 세입ᄌ...
연극 〈기형도 플레이〉,0.688852,연극 〈기형도 플레이〉.gif
연극 〈운빨로맨스〉- 대학로,0.670782,연극 〈운빨로맨스〉- 대학로.png
뮤지컬 〈레미제라블〉 - 부산,0.665092,뮤지컬 〈레미제라블〉 - 부산.png
국민 코믹 연극 〈오백에삼십〉 - 대학로 세우아트센터 1관,0.651022,국민 코믹 연극 〈오백에삼십〉 - 대학로 ...
뮤지컬 〈사칠〉,0.642459,뮤지컬 〈사칠〉.gif


In [ ]:
print(userid + " 님께 가장 추천하는 공연은 " + result.index[0] + "입니다")
print("👇 추천되는 공연 10가지와 점수 👇")

devil*** 님께 가장 추천하는 공연은 뮤지컬 〈타오르는 어둠 속에서〉입니다
👇 추천되는 공연 10가지와 점수 👇
